<a href="https://colab.research.google.com/github/nickwan/nwds-stream-notebooks/blob/main/pitch_printer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from IPython.display import HTML, display, clear_output
import warnings
import time

In [ ]:
year = 2023
project_dir = 'drive/My Drive/baseball/data'
df = pd.read_csv(f"{project_dir}/{year}-data.csv")
df.loc[df['p_throws']=='L', 'pfx_x'] = df.loc[df['p_throws']=='L', 'pfx_x'].mul(-1)
df.loc[:, ['pfx_x', 'pfx_z']] = df.loc[:, ['pfx_x', 'pfx_z']].mul(12)

rs = df['pitch_name'].dropna().drop_duplicates().shape[0]
pts = [
    'Forkball', 'Slider', 'Sweeper', 'Sinker', 'Screwball',
    '4-Seam Fastball', 'Knuckleball', 'Split-Finger', 'Changeup',
    'Other', 'Cutter','Pitch Out', 'Slow Curve', 'Eephus',
    'Slurve','Curveball', 'Knuckle Curve'
]
cmap = sns.palettes.color_palette('Spectral', n_colors=rs)
cmap = {pt:c for pt,c in zip(pts, cmap)}
hp_x = [0,-.9,-.7,.7,.9,0]
hp_y = [-.1,.1,.4,.4,.1,-.1]

In [ ]:
warnings.filterwarnings('ignore')

ids = df.loc[:, ['game_pk', 'inning', 'inning_topbot','at_bat_number']].sample(1000)
for _,id in ids.iterrows():
  try:
    gid = id['game_pk']
    iid = id['inning']
    hid = id['inning_topbot']
    aid = id['at_bat_number']
    df_plt = df.loc[(df['game_pk']==gid) & (df['inning']==iid) & (df['inning_topbot']==hid) & (df['at_bat_number']==aid)]

    sz_top, sz_bot = df_plt['sz_top'].mean(),df_plt['sz_bot'].mean()
    hue_order = df_plt['pitch_name'].unique()
    _pal = [cmap[pt] for pt in hue_order]
    title_str = f"""
    Pitcher: {df_plt['pitcher_name'].unique()[0]}
    Batter: {df_plt['batter_name'].unique()[0]}
    {df_plt['des'].unique()[0]}
    """
  except:
    continue

  for p in range(df_plt['pitch_number'].max()):
    time.sleep(2)
    _df_plt = df_plt.loc[df_plt['pitch_number']<=p+1].copy()
    fig = plt.figure(figsize=(7,7))
    fig = sns.scatterplot(data=_df_plt, x='plate_x', y='plate_z',
                          hue='pitch_name', hue_order=hue_order,
                          s=200, palette=_pal,
                          edgecolor='k')

    for idx,row in _df_plt.iterrows():
      color='k'
      if row['pitch_type'] in ['KC']:
        color='w'

      plt.text(row['plate_x'], row['plate_z'],row['pitch_number'],
              color=color, zorder=3, fontsize=12, ha='center', va='center')

    fig.set_xlim(-2,2)
    fig.set_ylim(-0.5,5)
    fig.plot([(-17/12)/2,(-17/12)/2,(17/12)/2,(17/12)/2,(-17/12)/2],
            [sz_bot,sz_top,sz_top,sz_bot,sz_bot], c='k', ls='-',
            zorder=4, alpha=.6)
    fig.plot(hp_x, hp_y, c='k', alpha=.4)
    fig.set_aspect('equal')
    fig.xaxis.set_visible(False)
    fig.yaxis.set_visible(False)
    plt.legend(bbox_to_anchor=(.8,.1), loc=2)
    sns.despine(left=True, bottom=True)
    plt.show()

    fig = plt.figure(figsize=(5,5))
    fig = sns.scatterplot(data=_df_plt, x='pfx_x', y='pfx_z',
                          hue='pitch_name', hue_order=hue_order,
                          s=150, palette=_pal, edgecolor='k',
                          legend=False)

    for idx,row in _df_plt.iterrows():
      color='k'
      if row['pitch_type'] in ['KC']:
        color='w'

      plt.text(row['pfx_x'], row['pfx_z'],row['pitch_number'],
              color=color, zorder=3, fontsize=12, ha='center', va='center')

    fig.set_xlim(-20,20)
    fig.set_ylim(-20,20)
    fig.set_aspect('equal')
    plt.axhline(0, ls=':', c='k')
    plt.axvline(0, ls=':', c='k')
    plt.xlabel('')
    plt.ylabel('')
    plt.title(title_str)
    sns.despine(left=True, bottom=True)
    plt.show()
    clear_output(wait=True)


KeyboardInterrupt: 